In [87]:
import pandas as pd
import numpy as np

In [88]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


#["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
#"https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1lm6ngv02kq97fprhbTa98lI3fI5U9Pr6dPweEkitZeo'
SAMPLE_RANGE_NAME = 'A:Z'
SHEET_ID = 0

# https://googleapis.github.io/google-api-python-client/docs/dyn/sheets_v4.spreadsheets.html
# API reference https://developers.google.com/sheets/api/reference/rest

In [89]:
def main():
    """Shows basic usage of the Sheets API.
        Prints values from a sample spreadsheet.
        """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return

        print('Name, Major:')
        for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
            print('%s, %s' % (row[0], row[4]))
    except HttpError as err:
        print(err)


In [90]:
main()

NameError: name 'SAMPLE_SPREADSHEET_ID' is not defined

In [91]:
def get_authenticated_sheets_service():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)
        print("Authenticated into google sheets")
        return service
    
    except HttpError as err:
        
        print(err)
        return
    

In [92]:
service = get_authenticated_sheets_service()

Authenticated into google sheets


In [108]:
def get_df_from_sheets(service, spreadsheet_id, range):
    result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id, range=SAMPLE_RANGE_NAME).execute()
    
    rows = result.get('values', [])
    print(f"{len(rows)} rows retrieved")
    df = pd.DataFrame(rows, columns = rows[0])
    df = df.iloc[1:]
    # Replace all None values with NaN
    df = df.fillna(value=np.nan)
    
    return df

In [112]:
result = service.spreadsheets().values().get(
            spreadsheetId=SPREADSHEET_ID, range="fact_videos").execute()
    
rows = result.get('values', [])
print(f"{len(rows)} rows retrieved")
print(rows[21][:])

22 rows retrieved
['21']


In [113]:
df = pd.DataFrame(rows, columns = rows[0])
df = df.iloc[1:]
# Replace all None values with NaN
df = df.fillna(value=np.nan)
df = df.replace(r'^\s*$', np.nan, regex=True)

# Existing transformation
df["post_date"] = pd.to_datetime(df["post_date"])
df["created_at"] = pd.to_datetime(df["created_at"])
df["uploaded_at"] = pd.to_datetime(df["uploaded_at"])
df.index = df["id"]

In [122]:
df.loc['10', "created_at"]

NaT

In [115]:
df

,id,post_date,created_at,uploaded_at,music_file,background,title,description,tags,text1,text2,text3,text4,text5,text6
id,,,,,,,,,,,,,,,
1,1,2023-01-01,2023-01-01 15:02:00,2023-01-01 15:39:00,blade_runner_synth.mp3,https://www.pexels.com/video/a-sunset-time-lap...,Fun facts you need to know #facts #shorts,Comment which one is your favorite.,"facts, crazy, cool, funny",Crazy facts you need to know (part 1),Potato chips cause more weight gain than any o...,Our brains make up on average 2% of our body w...,"If a bee consumes alcohol and gets drunk, he w...",NaN,NaN
2,2,2023-01-02,2023-01-02 14:54:00,2023-01-02 14:58:00,blade_runner_synth.mp3,https://www.pexels.com/video/top-view-of-sea-w...,Fun facts you need to know (part 2) #facts #sh...,Comment which one is your favorite.,"facts, fun, knowledge, funny",Crazy facts you need to know (part 2),"If you sneeze too hard, you could fracture a rib.",Wearing headphones for just an hour could incr...,"In the course of an average lifetime, while sl...",NaN,NaN
3,3,2023-01-02,2023-01-02 14:58:00,2023-01-02 14:58:00,blade_runner_synth.mp3,https://www.pexels.com/video/a-sunset-time-lap...,Fun facts you need to know #facts #shorts #funny,Comment which one is your favorite.,"facts, crazy, fun, funny, knowledge",Fun facts you need to know (part 1),Potato chips cause more weight gain than any o...,Our brains make up on average 2% of our body w...,"If a bee consumes alcohol and gets drunk, he w...",NaN,NaN
4,4,2023-01-03,2023-01-03 17:32:00,2023-01-03 17:36:00,blade_runner_synth.mp3,https://www.pexels.com/video/aerial-footage-of...,Crazy facts about your body (Part 1) #facts #s...,Comment which one is your favorite.,"facts, crazy, fun, funny, knowledge",Mind-blowing facts about the human body (part 1),"When you cry and your nose becomes runny, it’s...",Your brain blocks you from feeling your organs...,Your body can randomly decide to become allerg...,NaN,NaN
5,5,2023-01-04,2023-01-03 17:35:00,2023-01-03 17:36:00,blade_runner_synth.mp3,https://www.pexels.com/video/drone-footage-of-...,Fun facts you need to know (part 3) #facts #sh...,Comment which one is your favorite.,"facts, crazy, fun, funny, knowledge",Fun facts you need to know (part 3),The United States has lost 6 nuclear weapons o...,Making your bed everyday exposes bed mites to ...,"When a person receives a kidney transplant, th...",NaN,NaN
6,6,2023-01-04,2023-01-05 00:35:00,2023-01-03 17:36:01,paris.mp3,https://www.pexels.com/video/video-of-cascadin...,Crazy facts you need to know #facts #shorts #f...,Comment which one is your favorite.,"facts, crazy, fun, funny, knowledge",Crazy facts you need to know (part 3),"If a Polar Bear and a Grizzly Bear mate, their...",The only letter that doesn’t appear on the per...,There were two AI chatbots created by Facebook...,NaN,NaN
7,7,2023-01-05,2023-01-05 00:41:00,2023-01-05 00:59:00,paris_drill_trimmed.mp3,https://www.pexels.com/video/aerial-footage-of...,Fun facts you need to know (part 2) #facts #sh...,Comment which one is your favorite.,"facts, fun, knowledge, funny",Crazy facts you need to know (part 2),"If you sneeze too hard, you could fracture a rib.",Wearing headphones for just an hour could incr...,"In the course of an average lifetime, while sl...",NaN,NaN
8,8,2023-01-06,2023-01-06 18:49:00,2023-01-06 18:50:00,paris.mp3,https://www.pexels.com/video/footage-of-a-moun...,Fun facts you need to know (Part 4) 🤯 #facts #...,Comment which one is your favorite.,"facts, fun, knowledge, funny",Fun facts you should know (part 4),Your dog estimates the time of your return hom...,"Two hundred years ago, over 80% of the world l...","At birth, a baby panda is smaller than a mouse.",The ozone layer is slowly repairing itself.,NaN
9,9,2023-01-07,2023-01-06 18:35:00,2023-01-06 18:35:00,paris.mp3,https://www.pexels.com/video/aerial-footage-of...,Unfun facts you should know (Part 1) 😳 #facts ...,Comment which one is your favorite.,"facts, unfun, knowledge, scary",Scary facts you should probably know,"When it c

#### Now we need to write it back to the sheets



In [117]:
d = df.fillna("")

In [118]:
# Write the DataFrame to the sheet
values =  [list(d.columns)] + d.values.tolist()
values = [[str(v) for v in row] for row in values]
values

[['id',
  'post_date',
  'created_at',
  'uploaded_at',
  'music_file',
  'background',
  'title',
  'description',
  'tags',
  'text1',
  'text2',
  'text3',
  'text4',
  'text5',
  'text6'],
 ['1',
  '2023-01-01 00:00:00',
  '2023-01-01 15:02:00',
  '2023-01-01 15:39:00',
  'blade_runner_synth.mp3',
  'https://www.pexels.com/video/a-sunset-time-lapse-8035714/',
  'Fun facts you need to know #facts #shorts',
  'Comment which one is your favorite.',
  'facts, crazy, cool, funny',
  'Crazy facts you need to know (part 1)',
  'Potato chips cause more weight gain than any other food',
  'Our brains make up on average 2% of our body weight but consume 20% of our caloric intake',
  "If a bee consumes alcohol and gets drunk, he won’t be let back into the hive until he's sober.",
  '',
  ''],
 ['2',
  '2023-01-02 00:00:00',
  '2023-01-02 14:54:00',
  '2023-01-02 14:58:00',
  'blade_runner_synth.mp3',
  'https://www.pexels.com/video/top-view-of-sea-waves-crashing-on-seashore-6775423/',
  'Fun 

In [119]:
list(d.columns)

['id',
 'post_date',
 'created_at',
 'uploaded_at',
 'music_file',
 'background',
 'title',
 'description',
 'tags',
 'text1',
 'text2',
 'text3',
 'text4',
 'text5',
 'text6']

In [120]:

body = {
    'values': values
}

result = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range="fact_videos",
    valueInputOption='USER_ENTERED',#'RAW',
    body=body).execute()
print(f'{result["updatedCells"]} cells updated.')

330 cells updated.


In [ ]:
def update_sheet_from_df(service, df, sheet_name):
    d = df.fillna("")
    
    # Write the DataFrame to the sheet
    values =  [list(d.columns)] + d.values.tolist()
    values = [[str(v) for v in row] for row in values]
    
    body = {
        'values': values
    }

    result = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID, range="fact_videos",
        valueInputOption='USER_ENTERED',#'RAW',
        body=body).execute()
    print(f'{result["updatedCells"]} cells updated.')
    
    
